# Load Libraries :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import gaussian_filter, sobel

# 1 - Fundamental Level Sets : 

## Introduction 
Instead of representing the active contour explicitly by a set of ordered point, We define the contour implicitely by the zero level set of a function phi(x,y), by evolving this function in time we can approch our desired contour.  
During evolution:
- *ϕ* is updated over time using a **partial differential equation (PDE)**.
- The idea is that *ϕ* deforms under some speed function so that the zero-level set (the contour) moves toward the object boundary in the image.

## Mathematical Formulation

The basic level set method represents a contour 𝐶 implicitly as the zero level set of a higher-dimensional function ϕ(x,y,t), where:
- (x,y) are spatial coordinates
- t is time/iteration
- The contour C is defined where ϕ(x,y,t) = 0

The contour evolution is governed by the level set equation:

$$\frac{\partial \phi}{\partial t} + F|\nabla \phi| = 0$$

Where:
- F is the speed function that controls the contour evolution
- ∇ϕ is the spatial gradient of ϕ

## Basic Algorithm Components

### 1. Level Set Function Initialization

Typically, the level set function is initialized as a signed distance function:
- ϕ(x,y,0) < 0 for points inside the initial contour
- ϕ(x,y,0) > 0 for points outside
- |ϕ(x,y,0)| represents the distance to the contour

A common initialization is:
$$\phi(x,y,0) = \pm d$$

Where d is the distance from (x,y) to the initial contour, and the sign depends on whether the point is inside (negative) or outside (positive).

### 2. Speed Function Design

For image segmentation, the speed function F typically depends on:
- Image gradient: to stop the contour at edges
- Curvature: to maintain smoothness

A basic speed function:
$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$

Where:
- g is an edge detector function (e.g., g = 1/(1+|∇I|²))
- I is the image
- κ is the curvature of the level set
- c (optional) is a constant (balloon force)

### 3. Level Set Evolution

At each iteration, update ϕ using:
$$\phi^{n+1} = \phi^n - \Delta t \cdot F|\nabla \phi^n|$$


#### Time Step Selection

For stability, the time step must satisfy the CFL condition:
$$\Delta t \leq \frac{\Delta x}{max(|F|)}$$

## Complete Algorithm

```
1. Initialize φ(x,y,0) as a signed distance function
2. Calculate the edge indicator function g(|∇I|)
3. For each iteration n:
   a. Calculate ∇φⁿ using finite differences
   b. Compute curvature κ
   c. Evaluate speed function F
   d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
   e. Periodically reinitialize φ to maintain signed distance property
4. Extract final contour from zero level set φ = 0
```



## Implementation :

In [ ]:
# Load image
img = cv2.imread("coin.jpeg", cv2.IMREAD_GRAYSCALE)

plt.figure()
plt.imshow(img, cmap='gray')
plt.title('Input Image')
plt.axis('off')
plt.show()

### 1. Initialize the LSF : φ(x,y,0)

In [ ]:
# 1. Initialize the LSF : φ(x,y,0)
def initialize_level_set_circle(shape, center=None, radius=None):
    """
    Initializes a level set function phi as a signed distance function to a circle.

    Parameters:
    - shape: tuple of ints, (height, width) of the domain
    - center: tuple (cx, cy) for the center of the circle; defaults to image center
    - radius: int, radius of the initial circle

    Returns:
    - phi: 2D numpy array of shape `shape`, representing the level set function
    """
    h, w = shape
    if center is None:
        center = (w // 2, h // 2)

    if radius is None:
        radius = (h // 2) - (h // 20)

    x = np.arange(0, w)
    y = np.arange(0, h)
    X, Y = np.meshgrid(x, y)

    # Signed distance to the circle: negative inside, zero on boundary, positive outside
    phi_init = np.sqrt((X - center[0])**2 + (Y - center[1])**2) - radius

    # Normalize 
    phi_init = phi_init / (np.max(np.abs(phi_init)) + 1e-10)  # Normalize to [-1, 1]

    return phi_init

phi_init = initialize_level_set_circle(img.shape)

plt.figure()
plt.imshow(phi_init, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.contour(phi_init, levels=[0], colors='r')  # zero level set
plt.title('Initial LSF')
plt.axis('off')
plt.show()

### 2. Calculate the edge indicator function g(|∇I|)
$$ g(|\nabla I|) = \frac{1}{1 + |\nabla I|^2} $$  
This function is **small near strong edges**, causing the evolution to slow down and stop there

In [ ]:
# 2. Calculate the edge indicator function g(|∇I|)
def compute_edge_indicator(img, sigma=1.0):
    """
    Compute the edge indicator function g for level set evolution.
    
    Parameters:
    - img: 2D array, input grayscale image
    - sigma: float, standard deviation for Gaussian smoothing
    
    Returns:
    - g: 2D array, edge indicator function
    """
    # Convert image to float if not already
    img = img.astype(float)
    
    # Normalize image to [0,1] if needed
    if np.max(img) > 1.0:
        img = img / 255.0
    
    # Step 1: Apply Gaussian smoothing to reduce noise
    img_smooth = gaussian_filter(img, sigma)
    
    # Step 2: Compute gradients using Sobel operators
    grad_x = sobel(img_smooth, axis=1)
    grad_y = sobel(img_smooth, axis=0)
    
    # Step 3: Compute gradient magnitude
    grad_mag = np.sqrt(grad_x**2 + grad_y**2 + 1e-10)  # Small epsilon to avoid division by zero
    
    # Step 4: Compute edge indicator function (standard formulation)
    g = 1.0 / (1.0 + grad_mag**2)
    
    # Apply a light smoothing to the edge indicator to reduce artifacts
    g = gaussian_filter(g, sigma=0.5)
    
    return g

g = compute_edge_indicator(img)

plt.figure()
plt.imshow(g, cmap='gray')
plt.colorbar()
plt.title('edge indicator function g(|∇I|)')
plt.axis('off')
plt.show()

### 3. For each iteration n:
####    a. Calculate ∇φⁿ using finite differences  
- phi_x, phi_y for directional updates and geometric flow
- |∇φ| for evolving the zero level set in the normal direction

In [ ]:
# 3. For each iteration n:
#    a. Calculate ∇φⁿ using finite differences
def compute_phi_gradient(phi):
    """
    Compute the gradient of the level set function φ.

    Parameters:
    - phi: 2D array, the level set function

    Returns:
    - phi_x: derivative of φ in x direction (axis=1, columns)
    - phi_y: derivative of φ in y direction (axis=0, rows)
    - grad_phi_mag: gradient magnitude sqrt(φ_x^2 + φ_y^2)
    """
    grad_phi_y, grad_phi_x = np.gradient(phi)  # order: rows (y), columns (x)
    grad_phi_mag = np.sqrt(grad_phi_x**2 + grad_phi_y**2) + 1e-10  # small epsilon to avoid division by 0
    return grad_phi_x, grad_phi_y, grad_phi_mag
    
grad_phi_x, grad_phi_y, grad_phi_mag = compute_phi_gradient(phi_init)

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(grad_phi_x, cmap='gray')
plt.colorbar()
plt.title('grad_phi_x')

plt.subplot(1,3,2)
plt.imshow(grad_phi_y, cmap='gray')
plt.colorbar()
plt.title('grad_phi_y')

plt.subplot(1,3,3)
plt.imshow(grad_phi_mag, cmap='gray')
plt.colorbar()
plt.title('grad_phi_mag')

plt.tight_layout()
plt.show()

####    b. Compute curvature κ  

$$ \kappa = \nabla \cdot \left( \frac{\nabla \phi}{|\nabla \phi|} \right) $$  
This curvature term is useful for:
- **Smoothing** the contour
- Regularizing its shape during evolution

In [ ]:
def compute_curvature(phi):
    """
    Compute the curvature of the level set function phi.
    
    The curvature is defined as the divergence of the normalized gradient:
    κ = div(∇φ/|∇φ|)
    
    Parameters:
    - phi: 2D array, level set function
    
    Returns:
    - curvature: 2D array, curvature of the level set
    """
    # Compute the gradient of phi
    phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi)
    
    # Compute the second derivatives
    # For x direction
    phi_xx = np.zeros_like(phi)
    phi_xx[:, 1:-1] = (phi[:, 2:] - 2*phi[:, 1:-1] + phi[:, :-2])  # Central difference for second derivative
    
    # For y direction
    phi_yy = np.zeros_like(phi)
    phi_yy[1:-1, :] = (phi[2:, :] - 2*phi[1:-1, :] + phi[:-2, :])  # Central difference for second derivative
    
    # Mixed derivative (xy)
    phi_xy = np.zeros_like(phi)
    phi_xy[1:-1, 1:-1] = (phi[2:, 2:] - phi[2:, :-2] - phi[:-2, 2:] + phi[:-2, :-2]) / 4.0
    
    # Small epsilon to avoid division by zero
    epsilon = 1e-10
    
    # Compute curvature using the formula:
    # κ = (φxx*φy² - 2*φxy*φx*φy + φyy*φx²) / (|∇φ|³)
    numerator = phi_xx * phi_y**2 - 2 * phi_xy * phi_x * phi_y + phi_yy * phi_x**2
    denominator = (grad_phi_mag**3 + epsilon)
    
    curvature = numerator / denominator
    
    # Limit the curvature values to avoid instabilities
    curvature = np.clip(curvature, -1.0, 1.0)
    
    return curvature

curvature = compute_curvature(phi_init)

plt.figure()
plt.imshow(curvature, cmap='gray')
plt.colorbar()
plt.title('curvature')
plt.axis('off')
plt.show()



#### c. Evaluate speed function F  

$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$
This form is equivalent to:
$$ F = g \cdot \kappa + g_x \cdot \phi_x + g_y \cdot \phi_y $$

In [ ]:
def compute_speed_function(phi, g, alpha=0.2, beta=1.2):
    """
    Compute the speed function for level set evolution.
    
    This implements a typical speed function that combines:
    1. Curvature-based regularization term
    2. Constant balloon force term
    3. External edge-based force term
    
    Parameters:
    - phi: 2D array, current level set function
    - g: 2D array, edge indicator function
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - speed: 2D array, speed function for level set evolution
    """
    # Compute curvature
    curvature = compute_curvature(phi)
    
    # Compute gradient of phi
    phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi)
    
    # Compute gradient of edge indicator function g
    g_x = np.zeros_like(g)
    g_y = np.zeros_like(g)
    
    # Central differences for interior points
    g_x[:, 1:-1] = (g[:, 2:] - g[:, :-2]) / 2.0
    g_y[1:-1, :] = (g[2:, :] - g[:-2, :]) / 2.0
    
    # Dot product of ∇g and ∇φ/|∇φ|
    epsilon = 1e-10  # Avoid division by zero
    nx = phi_x / (grad_phi_mag + epsilon)  # Normal vector x-component
    ny = phi_y / (grad_phi_mag + epsilon)  # Normal vector y-component
    edge_attraction = g_x * nx + g_y * ny
    
    # Compute speed function components:
    # 1. Curvature term: g * κ (regularization)
    # 2. Balloon force: g * α (expansion/contraction)
    # 3. Edge attraction: β * edge_attraction (pulls toward edges)
    speed = g * (curvature + alpha) + beta * edge_attraction
    
    return speed

speed_function = compute_speed_function(phi_init, g)

plt.figure()
plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.title('Speed Function')
plt.axis('off')
plt.show()

#### d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
$$ \phi^{t+1} = \phi^t + \Delta t \cdot F |\nabla \phi| $$

#### e. Run iterative loop

In [ ]:
def level_set_contour_detection(image, nb_iter, center_initial=None, radius_initial=None, time_step=0.5, alpha=0.1, beta=1.2):
    """
    Perform level set evolution for contour detection.
    
    Parameters:
    - image: 2D array, grayscale image
    - nb_iter: int, number of iterations
    - center_initial: tuple, (x, y) coordinates of initial circle center
    - radius_initial: float, radius of initial circle
    - time_step: float, time step Δt
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - phi_n: 2D array, final level set function
    """

    iter_display = nb_iter//10

    # Initialize the LSF
    phi_init = initialize_level_set_circle(image.shape, center_initial, radius_initial)
    
    # Edge indicator function
    g = compute_edge_indicator(image)
    
    # Display inputs
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_init, levels=[0], colors='r')
    plt.title('Input Image and Initial LSF Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(g, cmap='gray')
    plt.title('Edge Indicator Function')
    plt.colorbar()
    plt.axis('off')
    
    plt.suptitle("Initial LSF and Input")
    plt.tight_layout()
    plt.show()
    
    # Initialize the loop
    phi_n = phi_init.copy()
    
    for i in range(nb_iter):
        # Compute ∇φ and |∇φ|
        phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi_n)
        
        # Compute speed function
        speed_function = compute_speed_function(phi_n, g, alpha=alpha, beta=beta)
        
        # Update φ
        phi_n = phi_n + time_step * speed_function * grad_phi_mag
        
        # Display every 10 iterations
        if ((i+1) % iter_display) == 0 :
            plt.figure(figsize=(20, 4))

            plt.subplot(1, 4, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 4, 2)
            plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Speed Function')
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.imshow(grad_phi_mag, cmap='gray', vmin=0, vmax=0.5)
            plt.colorbar()
            plt.title('grad_phi_mag')
            plt.axis('off')
            
            plt.subplot(1, 4, 4)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Iteration {i+1}")
            plt.tight_layout()
            plt.show()
    
    # Final display
    plt.figure(figsize=(20, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('iteration Contour')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Speed Function')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(grad_phi_mag, cmap='gray', vmin=0, vmax=0.5)
    plt.colorbar()
    plt.title('grad_phi_mag')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Level Set Function')
    plt.axis('off')
    
    plt.suptitle("Final Display")
    plt.tight_layout()
    plt.show()
    
    return phi_n

img = cv2.imread("coins.jpg", cv2.IMREAD_GRAYSCALE)
phi_n = level_set_contour_detection(img, 2000, time_step=0.5, alpha=0.1, beta=1.2)

## Conclusion:

- To prevent the evolution from diverging into artifacts across the level set function (LSF), the time step must be kept very small. However, this leads to significant computational cost.

- For the contour to evolve smoothly and accurately converge toward the edges, careful analysis and fine-tuning are required. In particular, selecting appropriate values for **α** (the weight of the balloon force that pushes the contour inward) and **β** (the weight of the edge attraction term that halts the evolution at boundaries) is highly dependent on the specific image.

- As iterations progress, the gradient magnitude |grad_phi| becomes increasingly steep, causing instabilities. This results in the formation of anomalies within the LSF, as well as irregular, unwanted contours near some inner edges.

# 2 - Fundamental Level Set with Reinitialization 

Over time, the level set function phi **deviates from a signed distance function**:
- Gradients become too steep or too flat
- Contour evolution becomes unstable

So we periodically **reinitialize** phi to be a signed distance function again, **while preserving the zero level set**.

We solve the **reinitialization PDE**:

$$ \frac{\partial \phi}{\partial \tau} = \text{sign}(\phi_0)(1 - |\nabla \phi|) $$

Until steady state. Here:
- phi_0 : is the level set function before reinitialization.
- tau : artificial time

This converges to a function with |grad_phi| = 1 (a signed distance function).

In [ ]:
import scipy.ndimage

def reinitialize_phi(phi_n):
    """
    Reinitialize the level set function to be a signed distance function (SDF).
    
    Parameters:
    - phi_n: 2D array, current level set function
    
    Returns:
    - phi_reinit: 2D array, reinitialized level set function
    """

    # 1. Inside mask: where phi_n < 0
    inside_mask = phi_n < 0

    # 2. Outside mask: where phi_n >= 0
    outside_mask = ~inside_mask

    # 3. Distance to the interface from inside
    dist_inside = scipy.ndimage.distance_transform_edt(inside_mask)

    # 4. Distance to the interface from outside
    dist_outside = scipy.ndimage.distance_transform_edt(outside_mask)

    # 5. Signed distance function
    phi_reinit = dist_outside - dist_inside

    return phi_reinit


In [ ]:
def level_set_contour_detection_fundamental_reinitialization(image, nb_iter, center_initial=None, radius_initial=None, time_step=0.5, alpha=0.1, beta=1.2, reinit_every = None):
    """
    Perform level set evolution for contour detection.
    
    Parameters:
    - image: 2D array, grayscale image
    - nb_iter: int, number of iterations
    - center_initial: tuple, (x, y) coordinates of initial circle center
    - radius_initial: float, radius of initial circle
    - time_step: float, time step Δt
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - phi_n: 2D array, final level set function
    """

    # inputs
    if reinit_every == None :
        reinit_every = nb_iter//5

    iter_display = nb_iter//10

    # Initialize the LSF
    phi_init = initialize_level_set_circle(image.shape, center_initial, radius_initial)
    
    # Edge indicator function
    g = compute_edge_indicator(image)
    
    # Display inputs
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_init, levels=[0], colors='r')
    plt.title('Input Image and Initial LSF Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(g, cmap='gray')
    plt.title('Edge Indicator Function')
    plt.colorbar()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Initialize the loop
    phi_n = phi_init.copy()
    
    for i in range(nb_iter):
        # Compute ∇φ and |∇φ|
        phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi_n)
        
        # Compute speed function
        speed_function = compute_speed_function(phi_n, g, alpha=alpha, beta=beta)
        
        # Update φ
        phi_n = phi_n + time_step * speed_function * grad_phi_mag
        
        # Display every 10 iterations
        if ((i+1) % iter_display) == 0 :
            plt.figure(figsize=(20, 4))

            plt.subplot(1, 4, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 4, 2)
            plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Speed Function')
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.imshow(grad_phi_mag, cmap='gray', vmin=0, vmax=0.5)
            plt.colorbar()
            plt.title('grad_phi_mag')
            plt.axis('off')
            
            plt.subplot(1, 4, 4)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Iteration {i+1}")
            plt.tight_layout()
            plt.show()

        if (i+1) % reinit_every == 0:
            phi_n = reinitialize_phi(phi_n)

            plt.figure(figsize=(10, 4))

            plt.subplot(1, 2, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Reinitialization : Iteration {i+1}")
            plt.tight_layout()
            plt.show()
    
    # Final display
    plt.figure(figsize=(20, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('iteration Contour')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Speed Function')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(grad_phi_mag, cmap='gray', vmin=0, vmax=0.5)
    plt.colorbar()
    plt.title('grad_phi_mag')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Level Set Function')
    plt.axis('off')
    
    plt.suptitle("Final Display")
    plt.tight_layout()
    plt.show()
    
    return phi_n

img = cv2.imread("coins.jpg", cv2.IMREAD_GRAYSCALE)

phi_n = level_set_contour_detection_fundamental_reinitialization(img, 2000, center_initial=None, radius_initial=None, time_step=0.5, alpha=0.1, beta=1.2, reinit_every = None)